## Ipsen perturbation bound - inducing sparsity in the original matrix

In the Ipsen SDP relaxation derivation, we start with the matrix to be perturbed (rank-one given by its factor rank_one_vector), then multiply it on the left and right by a diagonal matrix (inducing sparsity). This is relaxed to a Hadamard product with a PSD matrix S (sparsity-inducing), whose elements are in [0, 1].

COSMO might use an interior point method, but the gradient update and repeated solving acts as a sort of hopping around from simplex to simplex. It is still possible that at a certain simplex, the gradient does not allow us to move downhill to any other simplex. However, from a different starting condition, we might follow the gradient in a more favorable way.

In [1]:
using COSMO, LinearAlgebra, Random, CSV, Tables, Plots

In [2]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
# All close.
maximum(abs.(raw_data' * raw_data .- data))

4.884981308350689e-15

In [3]:
nodes = CSV.File("../NC-K7-Trace-Nodes.csv") |> Tables.matrix;
bounds = CSV.File("../NC-K7-Trace-Bounds.csv") |> Tables.matrix;
selected_node = sortperm(-(sum(nodes .== 1, dims = 1) .== 3)[1, :])[1]
bounds[:, selected_node]

3-element Array{Float64,1}:
 4.135540722357922
 5.870713667686698
 7.0

In [4]:
selected_data = [3; 83; 85];
# opt_inds obtained earlier through brute force.
opt_inds = [79; 80; 81; 84];
k = 7

7

In [5]:
# Optimal PCA.
maximum(eigen(data[[selected_data; opt_inds], :][:, [selected_data; opt_inds]]).values)

4.921536411066106

In [6]:
U = svd(raw_data[:, selected_data]).U[:, 1];
rank_one_vector = raw_data' * U;

In [7]:
residual_data = raw_data .- (U * U' * raw_data);

In [8]:
mapslices(norm, raw_data, dims=1)

1×101 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [9]:
mapslices(norm, residual_data, dims=1)

1×101 Array{Float64,2}:
 0.998223  0.986038  0.846852  0.949473  …  0.958106  0.996008  0.92525

In [10]:
# Chosen manually. Try using Hk helper function next.
greedy_z = [sortperm(abs.(rank_one_vector), rev=true)[1:6]; 3]

7-element Array{Int64,1}:
 84
 83
 85
 86
 80
 81
  3

In [29]:
z = 0. * zeros(n)
z[greedy_z] .= 1.
S = z * z';
S[3, 81]

1.0

In [41]:
# BRUTE-FORCE OPTIMUM initialization. Not suitable for actually evaluating.
z = 0. * zeros(n)
z[[selected_data; opt_inds]] .= 1.
S = z * z';
S[3, 81]

1.0

In [139]:
# Random initialization.
using StatsBase
z = 0. * zeros(n)
z[selected_data] .= 1.
while sum(z) < k
    z[sample(1:n)] = 1.
end
S = z * z';
S[3, 81]

1.0

In [12]:
raw_data_residual = raw_data - U * rank_one_vector';
data_residual = raw_data_residual' * raw_data_residual;

In [13]:
# data - data_residual >>= 0 (PSD test)
minimum(eigen(data - data_residual).values)

-1.325653942281207e-14

In [112]:
function SGradient(S)
    # Hadamard product and matrix-vector multiplication.
    vec = (S .* data_residual) * rank_one_vector;
    numer = norm(vec);
    denom = sqrt(sum(S .* rank_one_vector .* rank_one_vector'));
    (S .* rank_one_vector' .* vec ./ (numer * denom)) - (rank_one_vector .* rank_one_vector' * numer / 2 / denom^3)
end
function LinearObjective(S, S2)
    S2 = reshape(S2, n, n)
    linear_part = sum(Diagonal(rank_one_vector .^ 2) .* S2)

    vec = (S .* data_residual) * rank_one_vector;
    numer = norm(vec)
    # S already has a box constraint. Use max for safety.
    denom = sqrt(sum(max.(0, S) .* rank_one_vector .* rank_one_vector'))
    base_value = numer / denom
    grad = SGradient(S2)
    linear_part + base_value + sum(grad .* (S2 - S))
end

LinearObjective (generic function with 1 method)

In [15]:
maximum(abs.(SGradient(S)))

0.12086201071950316

In [16]:
sortperm(-abs.(vec(SGradient(S))))[1:10]

10-element Array{Int64,1}:
 8386
 8285
 8487
 8588
 7982
 8083
 8464
 8363
 8565
 8666

In [17]:
divrem(8386, 101)

(83, 3)

In [32]:
SGradient(S)[83, 3]

-0.005253719461733871

In [140]:
model = COSMO.Model()

psd_c = COSMO.Constraint(Matrix(1.0I, N, N), zeros(N), COSMO.PsdCone)
box_c = COSMO.Constraint(Matrix(1.0I, N, N), zeros(N), COSMO.Box(zeros(N), ones(N)))

# Tr S = k
trace_c = COSMO.Constraint(
    vec(Matrix(1.0I, n, n))',
    [-1. * k],
    COSMO.ZeroSet)

# One row: The diag entry may be zero or nonzero.
row_c = repeat(Matrix(-1.0I, n, n), inner=(1, n))
for i = 1:n
    row_c[i, 1 + (n+1)*(i-1)] = k-1
end
row_c = COSMO.Constraint(row_c, zeros(n), COSMO.ZeroSet)

rank_one_variance = rank_one_vector .^ 2

selected_c = zeros(n, n)
selected_c[selected_data[1], selected_data[1]] = 1.
selected_c[selected_data[2], selected_data[2]] = 1.
selected_c[selected_data[3], selected_data[3]] = 1.
selected_c = COSMO.Constraint(vec(selected_c)', -3., COSMO.ZeroSet)

assemble!(
    model,
    zeros(N, N),
    -vec(SGradient(S) + Diagonal(rank_one_vector .^ 2)),
    [psd_c, box_c, trace_c, row_c, selected_c],
    settings = COSMO.Settings(time_limit = 30, max_iter = 2500),
    x0 = vec(S))

In [141]:
result = COSMO.optimize!(model)
LinearObjective(S, result.x)

6.37270881378282

In [142]:
S = reshape(result.x, n, n);

In [143]:
update!(model, q = -vec(SGradient(S) + Diagonal(rank_one_vector .^ 2)))

In [144]:
result = COSMO.optimize!(model)
LinearObjective(S, result.x)

5.211448281925725

In [145]:
S = reshape(result.x, n, n);

In [146]:
update!(model, q = -vec(SGradient(reshape(result.x, n, n)) + Diagonal(rank_one_vector .^ 2)))

In [147]:
result = COSMO.optimize!(model)
LinearObjective(S, result.x)

5.211448812866272

In [258]:
z = 0. * zeros(n)
z[greedy_z] .= 1.
S = z * z';
S[3, 81]

1.0

In [105]:
# Random initialization.
using StatsBase
z = 0. * zeros(n)
z[selected_data] .= 1.
while sum(z) < k
    z[sample(1:n)] = 1.
end
S = z * z';
S[3, 81]

0.0

In [106]:
assemble!(
    model,
    zeros(N, N),
    -vec(SGradient(S) + Diagonal(rank_one_vector .^ 2)),
    [psd_c, box_c, trace_c, row_c, selected_c],
    settings = COSMO.Settings(time_limit = 30, max_iter = 10000),
    x0 = vec(S))

In [107]:
function RunLoop(S)
    for i = 1:5
        update!(model, q = -vec(SGradient(S) + Diagonal(rank_one_vector .^ 2)))
        result = COSMO.optimize!(model)
        print(LinearObjective(S, result.x))
        print("\n")
        S = reshape(result.x, n, n);
    end
end
RunLoop(S)

1.604039849684365


1.8175160354587083


2.3175221683233014


2.5229779675464377


2.369189301158896
